# OpenCV 人臉辨識

使用OpenCV的haarcascade模型進行人臉以及眼睛部位的辨別

haarcascades的模型可以從opencv 官方的github下載:
https://github.com/opencv/opencv/tree/master/data/haarcascades

In [1]:
import cv2
import sys
from PIL import Image
import os

In [2]:
def list_camera():
    """
    這個函數可以將電腦上所有可以用的 Camera都列出來
    回傳一個所有可用相機的 index list
    例如： [0]
    """
    index = 0
    cameras = []

    while True:
        cap = cv2.VideoCapture(index)
        
        if not cap.read()[0]:
            break
        else:
            cameras.append(index)

        cap.release()
        
        index += 1
    
    return cameras

In [3]:
def CatchUsbVideo(window_name, camera_idx):
    """ CatchUsbVideo 函式說明
    Args:
        window_name: 視窗Title的名稱
        camera_idx: 要使用的相機編號

    1. 開啟指定的相機，持續讀取影像
    2. 對讀取到的影像，判讀是否存在人臉物件
    3. 若有偵測到，則對每一個人臉物件畫出標記方框
    4. 接著裁取出人臉物件的影像，判讀眼睛物件
    5. 對眼睛物件進行畫出標記圓框
    
    若要中斷程序執行，請按 q 鍵即可
    """
    
    # 告訴OpenCV使用人臉識別分類器
    classfier_face = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
    classfier_eye = cv2.CascadeClassifier("haarcascade_eye.xml")
    
    cv2.namedWindow(window_name)

    # 開啟相機
    cap = cv2.VideoCapture(camera_idx)


    # 識別出人臉後要畫的邊框的顏色，BGR格式
    # OpenCV預設是 BGR 模式，不是 RGB 模式
    color_blue = (255, 0, 0)
    color_green = (0, 255, 0)
    color_red = (0, 0, 255)

    # 開啟相機，並持續的讀取相機所傳送進來的影像
    while cap.isOpened():

        ok, frame = cap.read() #讀取一幀資料
        
        if not ok:
            break

        # 將當前幀轉換成灰度影象
        # cv2.CascadeClassifier分類別只接受灰階影像
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 人臉檢測，1.2和2分別為圖片縮放比例和需要檢測的有效點數
        faceRects = classfier_face.detectMultiScale(grey, 
            scaleFactor = 1.2, 
            minNeighbors = 3,
            minSize = (32, 32))

        # faceRects 大於0，表示偵測到人臉，數值則是偵測到的人臉數目
        # 如果偵測到人臉物件，對人臉物體進行畫框標記
        if len(faceRects) > 0 :
                                                         
            for faceRect in faceRects:  #單獨框出每一張人臉
                # 取得該人臉物件的座標位址，以及長寬尺寸
                x, y, w, h = faceRect
                
                # 對人臉物件進行畫標記框
                # 可參照GT Wang 大大的文章，很清楚易懂
                # https://blog.gtwang.org/programming/opencv-drawing-functions-tutorial/
                # cv2.rectangle(影像內容, 頂點座標, 對向頂點座標, 顏色, 線條寬度)
                cv2.rectangle(frame, (x-10, y-10), (x+w, y+h), color_green, 2)                    
                
                # 只裁出該人臉物件的部份，可以減少運算量
                eye_grey = grey[y:y+h, x:x+w] # 灰度圖
                eye_color = frame[y:y+h, x:x+w] # 原始彩色圖

                # 把人臉的部份，偵測其眼睛物件
                eyeCircle = classfier_eye.detectMultiScale(eye_grey)
                
                # 把偵測到的眼睛畫圓框
                if len(eyeCircle) > 0:
                    for eyeCle in eyeCircle:
                        ex, ey, ew,eh = eyeCle
                        
                        # 先找出圓心點
                        center_of_circle = (int(ex+0.5*ew), int(ey+0.5*eh))
                        # 算出半徑
                        radius = int(ew*0.5)
                        # cv2.circle(影像, 圓心座標, 半徑, 顏色, 線條寬度)
                        cv2.circle(eye_color, center_of_circle, radius, color_red, 2)

        # 顯示影象
        cv2.imshow(window_name, frame)
        c = cv2.waitKey(10)
        
        # 按 q 鍵中斷
        if c & 0xFF == ord('q'):
            break

    #釋放攝像頭並銷燬所有視窗
    cap.release()
    cv2.destroyAllWindows()


In [4]:
# 找出自己環境上可用的相機
cameras = list_camera()
print(cameras)

[0]


In [ ]:
if len(cameras) > 0:
    # 指定要使用的相機
    camera = cameras[0]
    CatchUsbVideo("Age for face", camera)